In [1]:
import torch
device = torch.device('cuda')

In [2]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth')) # well trained road following model

<All keys matched successfully>

In [3]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [4]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224)

In [5]:
import ipywidgets

image_widget = ipywidgets.Image(format='jpeg', width=224, height=224)

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

camera.running = True

In [6]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

In [7]:

speed_control_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.001, description='speed control')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.04, description='steering gain')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_control_slider, steering_gain_slider, steering_bias_slider)

display(image_widget)


FloatSlider(value=0.0, description='speed control', max=1.0, step=0.001)

FloatSlider(value=0.04, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [8]:
import math
import time

go_on = 1
car.throttle = speed_control_slider.value

def execute(change):
    global robot, count_stops, go_on, x
    global steering_gain, steering_bias
                
    steering_gain = steering_gain_slider.value
    steering_bias = steering_bias_slider.value
       
    image_preproc = preprocess(change['new']).to(device)
    
    
    if go_on == 1:    
        car.throttle = speed_control_slider.value
        image = image_preproc.half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        x = float(output[0])
        car.steering = x * steering_gain + steering_bias            

            

execute({'new': camera.value})

In [9]:
camera.observe(execute, names='value')